In [1]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Authenticate with W&B
wandb.login(timeout=30)

# Configure project and other parameters if necessary
project_name = "contrastive_exploration"
entity = "pletctj6"

# Retrieve the runs from the project
api = wandb.Api()
runs = api.runs(f"{entity}/{project_name}")

# Initialize a list to store coverage and shannon entropy data
experiments_data = {}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pletctj6. Use `wandb login --relogin` to force relogin


In [2]:
for run in runs:
    # check status 
    if run.state != "finished":
        print(f"Skipping run {run.name} because it is not finished.")
        continue
    config = run.config
    exp_name = config.get('exp_name', 'unknown_exp')
    env_name = config.get('env_id', 'unknown_env')
    seed = config.get('seed', 'unknown_seed')
    keep_extrinsic_reward = config.get('keep_extrinsic_reward', False)
    beta_ratio = config.get('beta_ratio', None)
    use_sigmoid = config.get('use_sigmoid', None)
    feature_extractor = config.get('feature_extractor', None)
    history = run.history()
    # Retrieve the historical data for coverage, shannon entropy, and global step
    if "specific/coverage" not in history.columns or "specific/shanon_entropy" not in history.columns or "global_step" not in history.columns:
        print(f"Skipping run {exp_name} - {env_name} - {seed} because it doesn't have all the necessary data.")
        continue
    coverage_evolution = history['specific/coverage']
    shannon_entropy_evolution = history['specific/shanon_entropy']
    global_step_evolution = history['global_step']

    # Extract final values
    if "specific/coverage" not in run.summary or "specific/shanon_entropy" not in run.summary or "global_step" not in run.summary:
        print(f"Skipping run {exp_name} - {env_name} - {seed} because it doesn't have all the necessary data.")
        continue
    final_coverage = run.summary['specific/coverage']
    final_shannon_entropy = run.summary['specific/shanon_entropy']
    final_global_step = run.summary['global_step']

    # Initialize nested dictionaries if they don't exist
    if exp_name not in experiments_data:
        experiments_data[exp_name] = {}
    if env_name not in experiments_data[exp_name]:
        experiments_data[exp_name][env_name] = {}
    if seed not in experiments_data[exp_name][env_name]:
        experiments_data[exp_name][env_name][seed] = {
            "keep_extrinsic_reward": keep_extrinsic_reward,
            "coverage": coverage_evolution,
            "shannon_entropy": shannon_entropy_evolution,
            "final_coverage": final_coverage,
            "final_shannon_entropy": final_shannon_entropy,
            "global_step": global_step_evolution,
            "final_global_step": final_global_step,
            "beta_ratio": beta_ratio,
            "use_sigmoid": use_sigmoid, 
            "feature_extractor": feature_extractor,
            'config': config
        }


Skipping run smm_ppo - HumanoidStandup-v4 - 3 because it doesn't have all the necessary data.
Skipping run smm_ppo - HumanoidStandup-v4 - 5 because it doesn't have all the necessary data.
Skipping run smm_ppo - HumanoidStandup-v4 - 2 because it doesn't have all the necessary data.
Skipping run smm_ppo - HumanoidStandup-v4 - 1 because it doesn't have all the necessary data.
Skipping run smm_ppo - HumanoidStandup-v4 - 4 because it doesn't have all the necessary data.
Skipping run smm_ppo - Humanoid-v3 - 4 because it doesn't have all the necessary data.
Skipping run smm_ppo - Humanoid-v3 - 2 because it doesn't have all the necessary data.
Skipping run smm_ppo - Humanoid-v3 - 1 because it doesn't have all the necessary data.
Skipping run smm_ppo - Humanoid-v3 - 3 because it doesn't have all the necessary data.
Skipping run smm_ppo - Humanoid-v3 - 5 because it doesn't have all the necessary data.
Skipping run ngu_ppo - HumanoidStandup-v4 - 2 because it doesn't have all the necessary data.
S

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


CommError: HTTPSConnectionPool(host='api.wandb.ai', port=443): Read timed out. (read timeout=9)

In [1]:
print(experiments_data['smm_ppo']['Maze-Ur-v0'][1]['coverage'].shape)
smm_ppo_maze_ur_coverage = experiments_data['smm_ppo']['Maze-Ur-v0'][1]['coverage']
# check number of NaNs
print(np.sum(np.isnan(smm_ppo_maze_ur_coverage)))

# last global step
print(experiments_data['smm_ppo']['Maze-Ur-v0'][1]['final_global_step'])
print(experiments_data['smm_ppo']['Maze-Ur-v0'][2]['final_global_step'])

NameError: name 'experiments_data' is not defined

In [71]:
from jupyter_utils import dict_to_dataframe, is_not_empty
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# Fonction pour vérifier si une liste est vide
def is_not_empty(obj):
    if isinstance(obj, pd.Series):
        return not obj.empty
    return bool(obj)


def interpolate_and_fill_single_metric(values, global_steps, max_global_step):

    values = values.dropna()
    # Créer une série avec des NaNs pour les global_steps manquants
    all_steps = np.arange(max_global_step + 1)
    series = values.reindex(all_steps).interpolate(method='linear').ffill().bfill()
    print('series.shape', series.shape)
    print('nb nan', np.sum(np.isnan(series)))
    
    return series.to_numpy()  # Conversion de la série pandas en tableau NumPy

# Convertir le dictionnaire en DataFrame
def dict_to_dataframe(experiments_data, data_keys=['coverage', 'shannon_entropy', 'global_step']):
    records = []

    # Déterminer le global_step maximum pour chaque environnement
    max_global_steps = {}
    for exp_name in experiments_data.keys():
        for env_name in experiments_data[exp_name].keys():
            for seed in experiments_data[exp_name][env_name].keys():
                max_global_step = experiments_data[exp_name][env_name][seed]['final_global_step']
                if env_name not in max_global_steps:
                    max_global_steps[env_name] = max_global_step
                else:
                    max_global_steps[env_name] = max(max_global_steps[env_name], max_global_step)

    for exp_name in experiments_data.keys():
        for env_name in experiments_data[exp_name].keys():
            max_global_step = max_global_steps[env_name]
            for seed in experiments_data[exp_name][env_name].keys():
                data_global_step = experiments_data[exp_name][env_name][seed]['global_step']
                for data_key in data_keys:
                    data_raw = experiments_data[exp_name][env_name][seed][data_key]
                    data_filtered = interpolate_and_fill_single_metric(data_raw, data_global_step, max_global_step)
                            # print('data_interp', len(data_interp))
                # for all panda.core.series.Series in experiments_data[exp_name][env_name][seed]
                

    #             coverage = metrics.get('coverage', [])
    #             shannon_entropy = metrics.get('shannon_entropy', [])
    #             global_step = metrics.get('global_step', [])
    #             if is_not_empty(coverage) and is_not_empty(global_step):
    #                 coverage_interp = interpolate_and_fill_single_metric(coverage, global_step, max_global_step)
    #                 shannon_entropy_interp = interpolate_and_fill_single_metric(shannon_entropy, global_step, max_global_step)
    #                 global_step_full = list(range(max_global_step + 1))
                    
    #                 for c, s, g in zip(coverage_interp, shannon_entropy_interp, global_step_full):
    #                     records.append({
    #                         'exp_name': exp_name,
    #                         'env_name': env_name,
    #                         'seed': seed,
    #                         'coverage': c,
    #                         'shannon_entropy': s,
    #                         'global_step': g
    #                     })
    # return pd.DataFrame(records)

# Créer le DataFrame
df = dict_to_dataframe(experiments_data)

# # Déterminer les couleurs pour chaque algorithme
# unique_exp_names = df['exp_name'].unique()
# palette = plt.get_cmap("tab20").colors
# color_map = {exp_name: palette[i % len(palette)] for i, exp_name in enumerate(unique_exp_names)}

# # Plot pour chaque environnement
# environments = df['env_name'].unique()

# for env in environments:
#     try:
#         plt.figure(figsize=(12, 8))
#         env_df = df[df['env_name'] == env]
        
#         # Calculer la moyenne et l'écart-type pour chaque méthode et chaque étape
#         mean_std_df = env_df.groupby(['exp_name', 'global_step']).agg({
#             'coverage': ['mean', 'std']
#         }).reset_index()
#         mean_std_df.columns = ['exp_name', 'global_step', 'coverage_mean', 'coverage_std']
        
#         sns.set(style="whitegrid")
#         for exp in mean_std_df['exp_name'].unique():
#             exp_df = mean_std_df[mean_std_df['exp_name'] == exp]
#             print('coverage mean shape', exp_df['coverage_mean'].shape)
#             color = color_map[exp]
#             plt.plot(exp_df['global_step'], exp_df['coverage_mean'], label=f'{exp} mean', color=color)
#             plt.fill_between(exp_df['global_step'], 
#                              exp_df['coverage_mean'] - exp_df['coverage_std'], 
#                              exp_df['coverage_mean'] + exp_df['coverage_std'], 
#                              alpha=0.2, color=color)
        
#         plt.title(f'{env} Coverage')
#         plt.xlabel('Global Step')
#         plt.ylabel('Coverage')
#         plt.legend()
#         plt.show()
#     except Exception as e:
#         print(f"Failed to plot for environment {env} due to: {e}")